# PZQ-R field variants


We score and investigate variants from natural populations in order to have a comprehensive view of those that could impact praziquantel efficacy. We used data from different locations in New and Old World:
* Brazil (original from the associated publication)
* Senegal
* Niger
* Tanzania
* Uganda (data generated by Sanger https://doi.org/10.3389/fgene.2019.00826)
* Oman


## Environment and data

### Environment

Creating a conda environment improves reproducibility by installing specific versions of the programs used.

In [ ]:
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run or when the kernel is rebooted.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate PZQ-R_field

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

The cell below must be run only once at the time of the environment creation. The R script install the R dependencies.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

#### Phred/Phrap/Consed and PolyPhred

[Phred/Phrap/Consed](http://www.phrap.org/phredphrapconsed.html) and [PolyPhred](https://droog.gs.washington.edu/polyphred/) are software used to analyze Sanger sequencing traces by performing alignment and genotyping, respectively. They are freely available for academic used. However, this requires to contact the authors in order to get a copy ([here](http://www.phrap.org/consed/consed.html#howToGet) for Phred/Phrap/Consed, [here](https://droog.gs.washington.edu/polyphred/poly_get.html) for PolyPhred).

Once the copies obtained, they must be place in a download folder at the same level as this notebook. Then programs can be compiled.

In [ ]:
# Folder in which the software archives must be located
ddir="download"
[[ ! -d "$ddir" ]] && mkdir -p "$ddir"

##### Phred

Below is the procedure to compile phred and daev. This requires to have access to a terminal.

In [ ]:
myphred="phred-dist-020425.c-acd.tar.Z"

## Decompress archive
[[ ! -d "$ddir/phred" ]] && mkdir -p "$ddir/phred"
tar -C "$ddir/phred" -xzvf "$ddir/$myphred"

In [ ]:
# Copy the executables to the conda environment
cp -a "$ddir/phred/phred" "$ddir/phred/daev" "$CONDA_PREFIX/bin/"

# Copy the needed file
cp -a "$ddir/phred/phredpar.dat" "$CONDA_PREFIX/etc/phredpar.dat"

In [320]:
# Add the machine and chemistry used by the sequencing facility to the phredpar file
## Machine model and related information are available in the header of scf files
sed -i '/^begin/,/^# Chromatograms/ s/^#$/"KB_3730_POP7_BDTv3.mob"        terminator      big-dye                 ABI_3700\n#/' "$CONDA_PREFIX/etc/phredpar.dat"

(PZQ-R_field) (PZQ-R_field) 

: 1

##### Phd2fasta

Below is the procedure to compile phd2fasta. This requires to have access to a terminal.

In [ ]:
myphd="phd2fasta-acd-dist.130911.tar.gz"

## Decompress archive
[[ ! -d "$ddir/phd" ]] && mkdir -p "$ddir/phd"
tar -C "$ddir/phd" -xzvf "$ddir/$myphd"

In [ ]:
# Copy the executable to the conda environment
cp -a "$ddir/phd/phd2fasta" "$CONDA_PREFIX/bin/"

##### Phrap

Below is the procedure to compile phrap. This requires to have access to a terminal.

In [ ]:
myphrap="distrib.tar.Z"

## Decompress archive
[[ ! -d "$ddir/phrap.d" ]] && mkdir -p "$ddir/phrap.d"
tar -C "$ddir/phrap.d" -xzvf "$ddir/$myphrap"

In [ ]:
# Clean and copy the folder to the conda environment
rm "$ddir/phrap.d/"*.{c,h,o}
cp -aR "$ddir/phrap.d/" "$CONDA_PREFIX/bin/"

In [ ]:
ln -s "$CONDA_PREFIX/bin/phrap.d/"{cross_match,cross_match.manyreads,loco,phrap,phrap.longreads,phrap.manyreads,phrapview,swat} "$CONDA_PREFIX/bin/"

##### Consed

Below is the procedure to compile consed. This requires to have access to a terminal.

In [ ]:
mycs="consed_linux.tar.gz"

## Decompress archive
[[ ! -d "$ddir/consed" ]] && mkdir -p "$ddir/consed"
tar -C "$ddir/consed" -xzvf "$ddir/$mycs"

In [ ]:
for i in "$ddir/consed/"consed*
do
    ./"$i" -v
    [[ $? -eq 0 ]] && prog=$(basename "$i") && break
done

In [ ]:
# Correct small bug in install script (when whitespace is present in current path)
sed -i 's|cp $szDownloadDirectory|cp \\"$szDownloadDirectory\\"|g' "$ddir/consed/installConsed.perl"
sed -i 's|cp $szNewDir|cp \\"$szNewDir\\"|g' "$ddir/consed/installConsed.perl"

##### PolyPhred

Below is the procedure to compile polyphred. This requires to have access to a terminal.

In [ ]:
mypp="polyphred-6.18-binary-x86_64-unknown-linux-gnu.tar.gz"

## Decompress archive
tar -C "$CONDA_PREFIX/bin/" -xzvf "$ddir/$mypp"

# Link executable
ln -s "$CONDA_PREFIX/bin/polyphred-6.18-binary-x86_64-unknown-linux-gnu/bin/"* "$CONDA_PREFIX/bin/"

In [ ]:
# Correct perl location
sed -i '1s|^#!.*|#!/usr/bin/env perl|' "$CONDA_PREFIX/bin/polyphred-6.18-binary-x86_64-unknown-linux-gnu/bin/phredPhrap.pl"

##### Environment variable

In order to have phred and consed working as expected, two environment variables need to be exported each time a new Jupyter session is run or when the kernel is rebooted.

In [ ]:
export PHRED_PARAMETER_FILE="$CONDA_PREFIX/etc/phredpar.dat"
export CONSED_HOME="$CONDA_PREFIX/bin/consed.d"

### Sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

In [ ]:
# Bioproject
bioproject=ERP114942

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Keep S. mansoni samples only
fds=$(head -1 runinfo | tr "," "\n" | grep -w -n "ScientificName" | cut -d ":" -f 1)
awk -v fds=$fds -F "," '$fds == "Schistosoma mansoni"' runinfo > runinfo_sm

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir/$fln/" --split-files "$ldir/$fln/$run"
    rm "$ldir/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
        
done < runinfo_sm

# Compress files
pigz "$ldir/"*/*

rm runinfo*

Download our exome data

In [ ]:
# Bioproject
bioproject=PRJNA560069

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "SampleName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)
flk=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "download_path" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename, run and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    lnk=$(cut -d "," -f $flk <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    retry=0
    
    while [[ $retry -lt 2 ]]
    do
        # Download sra file
        wget -q -c -O "$ldir/$fln/$run" "$lnk"
        # Check integrity
        vdb-validate -q "$ldir/$fln/$run" &> /dev/null
        [[ $? -ne 0 ]] && ((retry++)) || break
    done
    
    # If max download attempt reached, issue message and move to the next
    [[ $retry -eq 2 ]] && echo "$run: dowloading problem" >> "$ldir/download_issue" && contine
    
    # Convert sra into fastq
    fastq-dump -O "$ldir/$fln/" --split-files "$ldir/$fln/$run"
    rm "$ldir/$fln/$run"
    
    # Rename file with more meaningful name
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo*

### Genome data

The genome data is downloaded from the [WormBase ParaSite](https://parasite.wormbase.org). We use the data from the version 14 (WBPS14). The data is then indexed for the different tools used.

In [ ]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir -p "$gdir"

# Download and unzip data
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

# Preparing indices
bwa index "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
samtools faidx "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

### Known variants

List of variants from the SmLE x SmHR crosses that segregated in a Mendelian fashion in F1 ([Valentim *et al.* 2013](https://doi.org/10.1126/science.1243106)). This list was generated from alignments using the v5 genome. This has been lifted over to the latest genome using the [flo pipeline](https://github.com/wurmlab/flo/tree/727f10b2b1c57a0514835d302d7f6345d3a34ffb).

In [ ]:
# Index file
gatk IndexFeatureFile -I "$gdir/sm_dbSNP_v7.vcf"

In [ ]:
# For Freebayes
bgzip -c "$gdir/sm_dbSNP_v7.vcf" > "$gdir/sm_dbSNP_v7.vcf.gz"
tabix "$gdir/sm_dbSNP_v7.vcf.gz"

### Population file

In [ ]:
find data/libraries/ -type d -name "*SAM*"   | sed "s|.*/||g" | sed "s|$|\tUganda|g" > data/pop
find data/libraries/ -type d -name "*Sm.TZ*" | sed "s|.*/||g" | sed "s|$|\tTanzania|g" >> data/pop
find data/libraries/ -type d -name "*Sm.SN*" | sed "s|.*/||g" | sed "s|$|\tSenegal|g" >> data/pop
find data/libraries/ -type d -name "*Sm.NE*" | sed "s|.*/||g" | sed "s|$|\tNiger|g" >> data/pop
find data/libraries/ -type d -name "*Sm.BR*" | sed "s|.*/||g" | sed "s|$|\tBrazil|g" >> data/pop
find data/libraries/ -type d -name "*Sm.OM*" | sed "s|.*/||g" | sed "s|$|\tOman|g" >> data/pop
#find data/libraries/ -type d -name *Sr* | sed "s|.*/||g" | sed "s|$|\tSro|g" >> data/pop

### GFF file

In [ ]:
# Gene code
mygene=Smp_246790
grep "$mygene" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" > "$gdir/$mygene.gff"

# Generate isoform specific gff
myiso=$(grep -o "$mygene\.[0-9]*" "$gdir/$mygene.gff" | sort | uniq)

for i in $myiso
do
    awk '$3 == "gene"' "$gdir/$mygene.gff" > "$gdir/$i.gff"
    grep -w "$i" "$gdir/$mygene.gff" | grep -v "exon" >> "$gdir/$i.gff"
done

## Sequencing data processing

### Snakemake pipeline

The snakemake pipeline performs the following steps:
* alignment of each library against the reference genome
* marking duplicates
* base quality recalibration
* generating alignment statistics
* outputing Smp_246790 gene read depth for each library
* outputing Z chromosome read depth for each library
* determining sex of each individual
* calling variants (gVCF) for each library
* combining gVCF of all libraries
* genotyping all samples per contig
* merging all the contigs to generate final VCF file

The VCF file produced will then be used to identify possible QTLs link to the cercarial production.

In [ ]:
# Status directory
statdir=status
[[ ! -d "$statdir" ]] && mkdir "$statdir"

snakemake --snakefile snakefile --cluster "qsub -V -cwd -o $statdir -j y -r y -pe smp 10 -S /bin/bash" --jobs 1000 -w 120

### Variant calling

We call the variants within the gene region across all samples directly from the BAM files using FreeBayes. This method is preferred because it can perform population defined calling.

In [ ]:
# Output directory
cdir="data/variant_calling"
[[ ! -d "$cdir" ]] && mkdir -p "$cdir"

# Variables
mygenome="$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
#mypos=SM_V7_3:600000-1000000
mypos=SM_V7_3:630000-800000

myfiles=$(find data -name *SAM*.bam -o -name *Sm*.bam | sort | tr "\n" " ") ## !! TO CHANGE SAMPLES

myfilename="$cdir/PZQ-R_field"


# Command used for calling variants
freebayes -f "$mygenome" \
   -r "$mypos" \
   -b $myfiles \
   -=          \
   --population data/pop \
   -q 20       \
   -m 30       \
   -@ "$gdir/sm_dbSNP_v7.vcf.gz" > "${myfilename}.vcf"
   
#     -F 0.4      \

pigz "${myfilename}".vcf

## Analyzing $SmTRP_{PZQ}$

We will analyze the previously called variants on the gene only. This analysis includes:
* Filtering out genotypes with low read depth,
* Removing invariable sites,
* Normalizing coordinates,
* Use a custom script to summarize variants and genotype from the VCF file.

In [ ]:
# Remove genotypes with low read depths
vcftools --gzvcf "${myfilename}.vcf.gz" \
    --minDP 4 \
    --recode  \
    --recode-INFO-all \
    --out "${myfilename}.flt-dp4"

# Rename file
mv "${myfilename}.flt-dp4.recode.vcf" "${myfilename}.flt-dp4.vcf"

# Clean the VCF of invariable ref variants
vcf-subset -a -e "${myfilename}.flt-dp4.vcf" > "${myfilename}.flt-dp4.cleaned.vcf"

# # Normalize VCF
# vt normalize -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
#     -r "$mygenome" \
#     "${myfilename}.flt-dp4.cleaned.vcf"
## This fails with a core-dump

## Maybe look at bcftools norm (with -f option)   !!! Install bcftools
bcftools norm -o "${myfilename}.flt-dp4.cleaned.norm.vcf" \
    -f "$mygenome" \
    "${myfilename}.flt-dp4.cleaned.vcf"

##Another method of normalization
# vcfleftalign -r "$mygenome" \
#     "${myfilename}.flt-dp4.cleaned.vcf" > "${myfilename}.flt-dp4.cleaned.norm.vcf"

In [ ]:
# Result folder
rdir="results/1-reports"
[[ ! -d "$rdir" ]] && mkdir -p "$rdir"

# Generate reports per isoform
for i in $myiso
do
    qsub -V -cwd -o "$statdir" -j y -b y scripts/func-table-mut.sh -v "${myfilename}.flt-dp4.cleaned.norm.vcf" \
        -r "$mygenome" \
        -g "$gdir/$i.gff" \
        -p data/pop \
        -o "$rdir/$i.flt.norm.tsv"
done

### Global coverage of the gene

We inspect how well the gene and particularly the exons are covered. This is done by concatenating individual coverage of the gene and then plotting total read depth at each position. The concatenation step can take ~20 minutes or more.

In [ ]:
fl_out=f

fl_ls=($(find data/libraries/ -type f -name "*PZQ.cov" | sort))

# Process first file
echo -e "#CHROM\t"$(cut -f1 "${fl_ls[0]}" | tr "\n" "\t" | sed "s/\t$//") > "$fl_out"
echo -e "POS\t"$(cut -f2 "${fl_ls[0]}"| tr "\n" "\t" | sed "s/\t$//") > "$fl_out"

for ((i = 0 ; i < ${#fl_ls[@]} ; i++))
do
echo $i
    echo -e $(dirname "${fl_ls[$i]}" | sed "s,.*/,,")"\t"$(cut -f 3 "${fl_ls[$i]}" | tr "\n" "\t" | sed "s/\t$//")  > "$fl_out"
done

### Mutations of interest

Some of the mutations are of high interest because located in critical sections of the protein (PZQ binding pocket, channel, TRP box). We list the samples that carry them in order to look at the reads within these samples to assess the likelihood of these mutations. This will also help identify samples that will require further investigation like PCR validation.

In [ ]:
# Result folder
rdir2="results/2-mutations of interest"
[[ ! -d "$rdir2" ]] && mkdir -p "$rdir2"

# mutations="p.T1394I p.Q1432H p.S1448Y p.G1458C p.H1522Q p.I1523N p.P1527T p.P1527Q p.Q1598K p.D1602Y p.D1606Y p.Q1609L p.Q1673K p.M1674I p.D1677Y p.H1680N p.P1683Q p.P1683R p.L1684I p.P1686T p.P1686Q p.P1687Q p.W1692L p.E1696* p.A1700D p.Q1704K"
mutations="p.H1522Q p.I1523N p.P1527T p.P1527Q p.Q1598K p.D1602Y p.D1606Y p.Q1609L p.Q1673K p.M1674I p.D1677Y p.H1680N p.P1686T p.P1686Q p.P1687Q"

# Add stop codon
mutations+=" $(awk '$8 ~ /*/ {print $8}' $rdir/Smp_246790.5.flt.norm.tsv)"

# Isoform of interest
myiso=Smp_246790.5

unset mylist
for m in $mutations
do
echo $m
    mypos=$(awk -v m=$m '$8 == m {print $1}' "$rdir/$myiso.flt.norm.tsv")
    myspl=$(awk -v mypos=$mypos '$2 == mypos' "${myfilename}.flt-dp4.cleaned.norm.vcf"| tr "\t" "\n" | grep -n 0/1 | cut -d ":" -f 1 | tr "\n" "," | sed "s/,$//")
    myspl=$(grep -m 1 "#C" "${myfilename}.flt-dp4.cleaned.norm.vcf" | cut -f $myspl)
    mylist+="$m\t$mypos\t$myspl""\n" 
done

echo -ne "$mylist" > "$rdir2/samples_list.tsv"

In [ ]:
# Output read depth for each sample carrying mutations of interest
Rscript scripts/variant_read_depth.R

Plots of the read depth for each mutation and sample are available in the folder labeled graphs.

### Primer design

Interesting mutations from the calling data must be checked by Sanger sequencing. These mutations are located in specific exons. To sequence the full exons, we extract exon sequences with an additional 200 bp on each end, where primers should be designed.

In [ ]:
fl_out="$rdir2/exons_of_interest"

myexons=(3 4 12 23 25 27 29 34)
myiso="Smp_246790.5"

grep "$myiso" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" | \
    awk '$3 == "CDS"'  | \
    sed -n "$(sed "s/ /p;/g" <<< ${myexons[@]})p" | \
    awk '{print $1"\t"$4-201"\t"$5+200}' > "$fl_out.bed"

bedtools getfasta -fi "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" \
    -bed "$fl_out.bed" \
    -fo "$fl_out.fas"

# Reformat fasta entry
myentries=($(grep -n ">" "$fl_out.fas" | cut -d ":" -f 1))
for ((i = 0 ; i < "${#myexons[@]}" ; i++))
do
    sed -i "${myentries[$i]}s/>/>E${myexons[$i]} /" "$fl_out.fas"
done

### Sanger sequencing

Sanger sequencing traces are analyzed using Consed and PolyPhred: The first align the data while the second perform the calling. PolyPhred output is then processed with a custom script in order to summarize the calling per sample.

In [ ]:
# Data folder
mkdir data/sanger_sequencing

# Download the data

In [ ]:
# Result folder
rdir3="results/2-mutations of interest/sanger_sequencing"
[[ ! -d "$rdir3" ]] && mkdir -p "$rdir3"

# Phred folders
mkdir -p "$rdir3/data/"{chromat_dir,edit_dir,phd_dir,poly_dir}

#### Reference sequence

In [ ]:
mygene="Smp_246790"

# Generate reference sequence
mkdir "$rdir3/0-Ref_seq"
bedtools getfasta -fi "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" \
    -bed <(grep "$mygene" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" | awk '$3 == "gene"' | awk '{print $1"\t"$4-1"\t"$5}') \
    -fo "$rdir3/Smp_246790.fas"

# Rename fasta entry
sed -i "s/>.*/>$mygene/" "$rdir3/Smp_246790.fas"

In [ ]:
pwd_old=$PWD

# Generate ref sequence for phred
cd "$rdir3/"
sudophred "$mygene.fas" -r -abi

# Move ref
mv "$mygene.REF" "data/chromat_dir/"
mv "$mygene.REF.phd.1" "data/phd_dir/"

cd "$pwd_old"

#### Prepare data

We create links to the original data but with modified file names in order to informed Polyphred about the paired samples. Samples will be identified on the first 8 characters

In [ ]:
# Generate modified sample names for polyphred
j=1
unset spl_code
for i in $(tail -n +2 "$rdir3/barcode_list.tsv" | cut -f 1 | sort | uniq)
do
    spl_code+="$i\t$(sed "s/^/$(printf "%02d" $j)_/" <<< "$i")\n"
    ((j++))
done

# Generate exon specific folder and link to data
while read line 
do
    # Sample name
    myspl=$(cut -f 1 <<< "$line")
    myspl=$(echo -ne "$spl_code" | grep "$myspl" | cut -f2)
    
    # Barcode
    mycode=$(cut -f 3 <<< "$line")
    [[ "$mycode" == NA ]] && continue
    
    # Link files
    ln -s "$PWD/data/sanger_sequencing/$mycode"*.scf "$rdir3/data/chromat_dir/${myspl}_$mycode.scf"

done < <(tail -n +2 "$rdir3/barcode_list.tsv")

Run Phred and Consed

In [ ]:
pwd_old=$PWD

export PHRED_PARAMETER_FILE=/master/fcheval/local/bin/consed_29.0/lib/phredpar.dat

# Moving to the working folder
cd "$rdir3/data/edit_dir"

# Generating phd and poly files
phred ../chromat_dir/*.scf -pd ../phd_dir/ -dd ../poly_dir/

# Preparing the reference sequence
phd2Ace.perl "$mygene.REF.phd.1"

# Aligning reads to the reference
find ../chromat_dir/* -name "*.scf" -exec basename {} \; > reads_to_add.fof
consed -ace "$mygene.REF.ace" -addNewReads reads_to_add.fof -newAceFilename "$mygene.REF.ace.1"
cd "$pwd_old"

Run PolyPhred

In [ ]:
score_snp=70 #90
score_indel=70 #80
qual=25 #40

# SNP scoring
name=$mygene.s${score_snp}-q$qual
polyphred -d "$rdir3/data" -extended_genotype -n $name.snp.nav -o $name.snp.polyphred.out -q $qual -refcomp -s 1 8 -score $score_snp

# Indel scoring
name=$mygene.s${score_indel}-q$qual
polyphred -d "$rdir3/data" -extended_genotype -i -inav $name.indel.nav -o $name.indel.polyphred.out -q $qual -s 1 8 -iscore $score_indel

In [ ]:
# Genotype extractions
scripts/trim_polyphred.sh -i "$rdir3/data/edit_dir/$mygene.s${score_snp}-q$qual.snp.polyphred.out" \
    -o "$rdir3/$mygene.SNP.gt.tsv" \
    -p 2 3 \
    -d "_"

scripts/trim_polyphred.sh -i "$rdir3/data/edit_dir/$mygene.s${score_indel}-q$qual.indel.polyphred.out" \
    -o "$rdir3/$mygene.INDEL.gt.tsv" \
    -p 2 3 \
    -d "_"
